In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors

### "Data Cleaning"

Source - the data origin

dna - DNA Sequence

zf - number of zinc fingers in protein

f1-fn - sequences of corresponding zinc finger regions

In [2]:
with open('database.txt', 'r') as file:
    data = file.read()

In [3]:
data = data.replace("source", "")
data = data.replace("dna", "")
data = data.replace("zf", "")
data = data.replace('f1', "")
data = data.replace("f2", "")
data = data.replace("f3", "")
data = data.replace("=", "")

In [4]:
z = data.split("\n")

In [5]:
textfile = open("database2.txt", "w")
for e in z:
    textfile.write(e + "\n")
textfile.close()

In [6]:
df = pd.read_fwf("database2.txt")

In [7]:
df.to_csv("output.csv", header=['Source', 'Dna', 'zf', 'f1', 'f2', 'f3', 'ex'])

## Hopefully simulating what they had in the paper

Sources:

http://www.cryst.bbk.ac.uk/education/AminoAcid/the_twenty.html

For Contacts:

01 - between amino acids a6 and nucleotide b1


02 - between amino acids a3 and nucleotide b2


03 - between amino acids a-1 and nucleotide b3


04 - between amino acids a2 and nucleotide b4



So this makes a canonical zinc finger binding model, to map each Zinc finger-DNA contact to a feature number. The contact positions are numbered from the start of the alpha-helix. 

This model is used to represent each protein-DNA complex 

In [8]:
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
base = ['a', 'c', 'g', 't']
contacts = ['01', '02', '03', '04']

In [9]:
s = []
for i in contacts:
    for j in amino_acids:
        for k in base:
            pair = i + j + k
            s.append(pair)

In [10]:
len(s)

320

Reading in the csv for the newly created database file

In [11]:
new_data = pd.read_csv('output.csv')
new_data

,Unnamed: 0,Source,Dna,zf,f1,f2,f3,ex
0,0,DBSFB01,ctcgcgGAAgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
1,1,DBSFB01,ctcgcgGCGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
2,2,DBSFB01,ctcgcgGTTgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
3,3,DBSFB01,ctcgcgGGGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex> 2ctcgcgGACgcggcc
4,4,DBSFB01,ctcgcgGGGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex> 2ctcgcgGATgcggcc
...,...,...,...,...,...,...,...,...
4077,4077,WYB95 c,tgcgTGGgcgccc 3,R,DELTRHIRI R,GNYTTHIRT R,DERKRHTKI e,Kd Kd20.0
4078,4078,WYB95 c,tGCGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT R,DERKRHTKI e,Kd Kd6.5
4079,4079,WYB95 c,tCTGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT R,DERKRHTKI e,Kd Kd101.0
4080,4080,WYB95 c,tGCGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT S,GQWWRHTKI e,Kd Kd13.1


In [12]:
new_data

,Unnamed: 0,Source,Dna,zf,f1,f2,f3,ex
0,0,DBSFB01,ctcgcgGAAgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
1,1,DBSFB01,ctcgcgGCGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
2,2,DBSFB01,ctcgcgGTTgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
3,3,DBSFB01,ctcgcgGGGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex> 2ctcgcgGACgcggcc
4,4,DBSFB01,ctcgcgGGGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex> 2ctcgcgGATgcggcc
...,...,...,...,...,...,...,...,...
4077,4077,WYB95 c,tgcgTGGgcgccc 3,R,DELTRHIRI R,GNYTTHIRT R,DERKRHTKI e,Kd Kd20.0
4078,4078,WYB95 c,tGCGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT R,DERKRHTKI e,Kd Kd6.5
4079,4079,WYB95 c,tCTGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT R,DERKRHTKI e,Kd Kd101.0
4080,4080,WYB95 c,tGCGtgggcgccc 3,R,DELTRHIRI R,DHLTTHIRT S,GQWWRHTKI e,Kd Kd13.1


In [13]:
positiveExamples = new_data.loc[(new_data['ex'] == "ex+") | (new_data['ex'] == "+")]
negativeExamples = new_data.loc[(new_data['ex'] == "ex-") | (new_data['ex'] == "-")]

In [14]:
print(len(positiveExamples))
print(len(negativeExamples))

98
689


#### Concatenation of all of the positive sets and negatives sets

In [15]:
updated_data = pd.concat([positiveExamples, negativeExamples])

In [16]:
updated_data

,Unnamed: 0,Source,Dna,zf,f1,f2,f3,ex
27,27,DBSFB01,ctcgatAAAgcggcc,3,KSADLKRHIRI,QRANLRAHIRT,TSGNLVRHTKI,ex+
28,28,DBSFB01,ctcgatAACgcggcc,3,KSADLKRHIRI,DSGNLRVHIRT,TSGNLVRHTKI,ex+
29,29,DBSFB01,ctcgatAAGgcggcc,3,KSADLKRHIRI,RSDTLSNHIRT,TSGNLVRHTKI,ex+
30,30,DBSFB01,ctcgatAATgcggcc,3,KSADLKRHIRI,TTGNLTVHIRT,TSGNLVRHTKI,ex+
31,31,DBSFB01,ctcgatACAgcggcc,3,KSADLKRHIRI,SPADLTRHIRT,TSGNLVRHTKI,ex+
...,...,...,...,...,...,...,...,...
3669,3669,CK94b t,tatagcgGCTgcgta,a,a 3 RSDELTR,IR NGGNLGRH,K RSDERKRHT,ex-
3670,3670,CK94b t,tatagcgGACgcgta,a,a 3 RSDELTR,IR NGGNLGRH,K RSDERKRHT,ex-
3673,3673,CK94b t,tatagcgGCTgcgta,a,a 3 RSDELTR,IR DRSNLERH,R RSDERKRHT,ex-
3676,3676,CK94b t,tatagcgGCTgcgta,a,a 3 RSDELTR,IR QRASLASH,R RSDERKRHT,ex-


#### Experimental Setup for the 80 categories

In [17]:
#creating the 80 categories
categories = []
for i in amino_acids:
    for j in base:
        pair = i + j
        categories.append(pair)

In [18]:
len(categories)

80

In [19]:
features = pd.get_dummies(categories, columns=['categories'])
features

,Aa,Ac,Ag,At,Ca,Cc,Cg,Ct,Da,Dc,...,Vg,Vt,Wa,Wc,Wg,Wt,Ya,Yc,Yg,Yt
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [20]:
#creating the 80 categories
all_categories = []
for k in contacts:
    for i in amino_acids:
        for j in base:
            pair = k + i + j
            all_categories.append(pair)

In [21]:
len(all_categories)

320

## SVM

In [31]:
#trying out labelEncoder for the labels N X B
le = LabelEncoder()
y_train_c = le.fit_transform(categories)
y_test_c = le.transform(categories)
print(y_train_c)
print(y_test_c)

[ 0  1  2  3 56 57 58 59 44 45 46 47  8  9 10 11  4  5  6  7 52 53 54 55
 12 13 14 15 20 21 22 23 24 25 26 27 28 29 30 31 36 37 38 39 32 33 34 35
 40 41 42 43 16 17 18 19 48 49 50 51 60 61 62 63 64 65 66 67 72 73 74 75
 76 77 78 79 68 69 70 71]
[ 0  1  2  3 56 57 58 59 44 45 46 47  8  9 10 11  4  5  6  7 52 53 54 55
 12 13 14 15 20 21 22 23 24 25 26 27 28 29 30 31 36 37 38 39 32 33 34 35
 40 41 42 43 16 17 18 19 48 49 50 51 60 61 62 63 64 65 66 67 72 73 74 75
 76 77 78 79 68 69 70 71]


In [54]:
#base model for proof of concept
X_train, X_test, y_train, y_test = train_test_split(features, categories)

print(X_train.shape[0])
print(X_test.shape[0])
print(len(y_train))
print(len(y_test))
print(X_train)
print(X_test)
print(y_train)
print(y_test)


#null model to just test things out 
zeros_train = np.zeros(X_train.shape)
zeros_test = np.zeros(X_test.shape)


base = svm.SVC(kernel='poly')
base.fit(zeros_train, y_train)
y_pred = base.predict(X_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

60
20
60
20
    Aa  Ac  Ag  At  Ca  Cc  Cg  Ct  Da  Dc  ...  Vg  Vt  Wa  Wc  Wg  Wt  Ya  \
61   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1    0   1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
71   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   1   0   
70   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   1   0   0   
21   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
45   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
19   0   0   0   0   0   0   0   1   0   0  ...   0   0   0   0   0   0   0   
78   0   0   0   0   0   0   0   0   0   0  ...   1   0   0   0   0   0   0   
13   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
44   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
32   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
75   0   0   0   0   0   0   0   0   0  

/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


#### More work needs to be done on SVM .. look in notes

## Random Forrest Classifier

In [52]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf_predict = clf.predict(X_test)
print(clf_predict)
print("Accuracy: ", metrics.accuracy_score(y_test, clf_predict))

[47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47]
Accuracy:  0.0


## Nearest Neighbors 

In [53]:
nbrs = NearestNeighbors()
nbrs.fit(X_train, y_train)
distances, indices = nbrs.kneighbors(X_test)
print(distances, indices)

[[1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1.41421356 1.41421356 1.41421356 1.41421356]
 [1.41421356 1

### MLP Classifier 

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [27]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(5,2),
                        max_iter = 10000,activation = 'relu',
                        solver = 'adam')

In [28]:
mlp_clf.fit(zeros_train, y_train)

MLPClassifier(hidden_layer_sizes=(5, 2), max_iter=10000)

In [29]:
y_pred = mlp_clf.predict(zeros_test)

In [30]:
accuracy_score(y_test, y_pred)

0.0